In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [2]:
#Image size
img_size = (128, 128)
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)



In [3]:
skin=r"dermet"

skin_1=ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_skin = skin_1.flow_from_directory(
    skin,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 15013 images belonging to 23 classes.


In [4]:
test_skin = skin_1.flow_from_directory(
    skin,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3743 images belonging to 23 classes.


In [5]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(128,128,3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(train_skin.num_classes, activation="softmax")   # Output = number of classes
])


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-08 11:17:08.251289: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-08 11:17:08.251322: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-09-08 11:17:08.251325: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-09-08 11:17:08.251563: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-08 11:17:08.251578: I tensorflow/core/common_runtime/pluggable_device/plu

In [6]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 23)             │         2,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,307,607 (12.62 MB)

 Trainable params: 3,307,607 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [10]:
# Train
history = model.fit(
    train_skin,
    epochs=20,
    validation_data=test_skin
)


Epoch 1/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 33s 68ms/step - accuracy: 0.1635 - loss: 2.9340 - val_accuracy: 0.1630 - val_loss: 2.9086
Epoch 2/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1613 - loss: 3.0788 - val_accuracy: 0.1811 - val_loss: 2.9175
Epoch 3/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1481 - loss: 3.3937 - val_accuracy: 0.1656 - val_loss: 3.0927
Epoch 4/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1555 - loss: 3.3872 - val_accuracy: 0.1627 - val_loss: 3.0011
Epoch 5/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1527 - loss: 3.4532 - val_accuracy: 0.1683 - val_loss: 3.1189
Epoch 6/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1508 - loss: 3.7947 - val_accuracy: 0.1688 - val_loss: 3.2947
Epoch 7/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1370 - loss: 4.7887 - val_accuracy: 0.1186 - val_loss: 3.4997
Epoch 8/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.1375 - loss: 4.9689 - 

In [11]:
# Save model
model.save("skin_care_assistant_model.h5")

In [13]:
skin_diseases = {
    "Acne and Rosacea": {
        "Overview": "Acne involves clogged pores and inflammation; Rosacea is chronic redness/flushing of the face.",
        "Causes": "Excess oil, bacteria, hormones (acne); blood vessel dilation, heat, alcohol, spicy food (rosacea).",
        "Symptoms": "Pimples, blackheads, whiteheads (acne); redness, visible blood vessels, bumps (rosacea).",
        "Care": "Use salicylic acid cleanser, benzoyl peroxide cream, topical antibiotics/retinoids (doctor), avoid oily food & alcohol, apply gentle sunscreen."
    },
    "Actinic Keratosis and Skin Cancers": {
        "Overview": "Precancerous or cancerous skin growths caused by sun damage.",
        "Causes": "UV radiation, tanning beds, fair skin, older age.",
        "Symptoms": "Scaly patches, non-healing sores, pearly/raised bumps.",
        "Care": "See dermatologist, topical chemotherapy (5-FU), immunotherapy creams, cryotherapy, surgical removal, SPF 50+ sunscreen."
    },
    "Atopic Dermatitis (Eczema)": {
        "Overview": "Chronic inflammatory skin condition with flare-ups.",
        "Causes": "Genetics, immune dysfunction, allergens, climate.",
        "Symptoms": "Dry, itchy, red patches, flare-ups.",
        "Care": "Moisturizers, hydrocortisone cream (doctor), lukewarm baths, avoid harsh soaps/allergens."
    },
    "Bullous Diseases": {
        "Overview": "Autoimmune blistering diseases (e.g., pemphigus, bullous pemphigoid).",
        "Causes": "Immune system attacks skin proteins.",
        "Symptoms": "Large fluid-filled blisters, fragile skin.",
        "Care": "Dermatologist care, corticosteroids, immunosuppressants, gentle skin care, avoid friction."
    },
    "Cellulitis and Bacterial Infections": {
        "Overview": "Bacterial infections of skin and deeper tissues.",
        "Causes": "Staphylococcus or Streptococcus bacteria.",
        "Symptoms": "Red, swollen, painful skin; pus or yellow crusts.",
        "Care": "Antibiotics (doctor), mupirocin cream, clean area, good hygiene."
    },
    "Exanthems and Drug Eruptions": {
        "Overview": "Rashes caused by viral infections or drug reactions.",
        "Causes": "Viruses, antibiotics, anticonvulsants, NSAIDs.",
        "Symptoms": "Widespread red rashes, itching, fever.",
        "Care": "Stop suspected drug, antihistamines, moisturizers, steroids if severe."
    },
    "Hair Loss (Alopecia)": {
        "Overview": "Hair thinning or bald patches.",
        "Causes": "Genetics, autoimmune, stress, nutrient deficiency.",
        "Symptoms": "Patchy bald spots, thinning hair.",
        "Care": "Minoxidil (doctor), protein-rich diet, biotin/vitamin D, stress management, PRP therapy."
    },
    "Herpes and HPV": {
        "Overview": "Viral STDs affecting skin.",
        "Causes": "HSV virus, HPV strains.",
        "Symptoms": "Blisters (herpes), warts (HPV).",
        "Care": "Antivirals (acyclovir), safe sex, avoid contact during outbreaks, HPV vaccines."
    },
    "Pigmentation Disorders": {
        "Overview": "Skin color imbalance (melasma, vitiligo).",
        "Causes": "Sun exposure, genetics, hormones, autoimmune.",
        "Symptoms": "Light/dark patches, photosensitivity.",
        "Care": "Sunscreen SPF 50+, hydroquinone/azelaic acid (doctor), avoid tanning, laser therapy."
    },
    "Lupus and Connective Tissue Diseases": {
        "Overview": "Autoimmune diseases affecting skin and organs.",
        "Causes": "Immune system attacking own tissues.",
        "Symptoms": "Butterfly rash, joint pain, fatigue.",
        "Care": "See rheumatologist, corticosteroids, immunosuppressants, sunscreen."
    },
    "Melanoma and Moles": {
        "Overview": "Skin cancer risk from abnormal moles.",
        "Causes": "UV damage, mutations.",
        "Symptoms": "Irregular moles (ABCDE rule: Asymmetry, Border, Color, Diameter, Evolution).",
        "Care": "Dermatologist check, surgical removal, sunscreen, early detection critical."
    },
    "Nail Fungus": {
        "Overview": "Fungal infection of nails.",
        "Causes": "Dermatophytes, yeast, poor hygiene.",
        "Symptoms": "Thick, brittle, yellow nails.",
        "Care": "Antifungal creams/nail lacquers, oral antifungals (doctor), trim nails, avoid sharing tools."
    },
    "Contact Dermatitis": {
        "Overview": "Allergic or irritant skin reaction.",
        "Causes": "Poison ivy/oak, detergents, chemicals.",
        "Symptoms": "Itchy rash, blisters, redness.",
        "Care": "Wash skin, calamine lotion, hydrocortisone, antihistamines."
    },
    "Psoriasis and Lichen Planus": {
        "Overview": "Chronic autoimmune skin disorders.",
        "Causes": "Overactive immune system.",
        "Symptoms": "Red plaques with silvery scales (psoriasis); purple bumps (lichen planus).",
        "Care": "Moisturizers, corticosteroid creams, phototherapy, biologics (doctor)."
    },
    "Scabies and Lyme Disease": {
        "Overview": "Parasitic infestations (scabies mites, tick bites).",
        "Causes": "Mites (scabies), Borrelia bacteria (Lyme).",
        "Symptoms": "Intense itching, burrows in skin (scabies), bullseye rash (Lyme).",
        "Care": "Permethrin cream, antibiotics for Lyme, wash bedding/clothes in hot water."
    },
    "Seborrheic Keratoses": {
        "Overview": "Benign, non-cancerous growths.",
        "Causes": "Aging, genetics.",
        "Symptoms": "Brown, waxy, wart-like spots.",
        "Care": "Usually harmless; cryotherapy/laser removal if irritating."
    },
    "Systemic Diseases (Skin Signs)": {
        "Overview": "Skin changes due to internal diseases.",
        "Causes": "Diabetes, thyroid issues, liver disease.",
        "Symptoms": "Dark patches, yellowing, dryness.",
        "Care": "Treat underlying condition, skincare, sunscreen."
    },
    "Fungal Infections (Tinea, Candidiasis)": {
        "Overview": "Fungal infections of skin, scalp, nails.",
        "Causes": "Dermatophytes, Candida yeast.",
        "Symptoms": "Ring-shaped rash, itching, scaling.",
        "Care": "Clotrimazole/terbinafine cream, keep dry, oral antifungals if severe."
    },
    "Urticaria (Hives)": {
        "Overview": "Allergic skin reaction.",
        "Causes": "Food, meds, stress, infections.",
        "Symptoms": "Raised itchy welts, redness.",
        "Care": "Antihistamines, cool compress, avoid triggers."
    },
    "Vascular Tumors": {
        "Overview": "Abnormal blood vessel growths (hemangiomas).",
        "Causes": "Genetics, childhood development.",
        "Symptoms": "Red/blue raised lesions.",
        "Care": "Usually benign; laser therapy, propranolol (infants), surgery if needed."
    },
    "Vasculitis": {
        "Overview": "Inflammation of blood vessels.",
        "Causes": "Autoimmune disease, infections, medications.",
        "Symptoms": "Red/purple spots, skin ulcers, fever, joint pain.",
        "Care": "Corticosteroids, immunosuppressants, medical care."
    },
    "Warts and Molluscum": {
        "Overview": "Viral skin infections.",
        "Causes": "HPV (warts), molluscum contagiosum virus.",
        "Symptoms": "Raised bumps (warts), pearly nodules (molluscum).",
        "Care": "Salicylic acid, cryotherapy, curettage, usually self-resolving."
    }
}

In [14]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load trained CNN model
model = tf.keras.models.load_model("skin_care_assistant_model.h5")

# Class labels
class_labels = ["Acne and Rosacea Photos", "Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions", "Atopic Dermatitis Photos", "Bullous Disease Photos","Cellulitis Impetigo and other Bacterial Infections", "Eczema Photos", "Exanthems and Drug Eruptions", "Hair Loss Photos Alopecia and other Hair Diseases", "Herpes HPV and other STDs Photos", "Light Diseases and Disorders of Pigmentation", "Lupus and other Connective Tissue diseases", "Melanoma Skin Cancer Nevi and Moles", "Nail Fungus and other Nail Disease", "Poison Ivy Photos and other Contact Dermatitis","Psoriasis pictures Lichen Planus and related diseases", "Scabies Lyme Disease and other Infestations and Bites", "Seborrheic Keratoses and other Benign Tumors","Systemic Disease", "Tinea Ringworm Candidiasis and other Fungal Infections", "Urticaria Hives", "Vascular Tumors", "Vasculitis Photos", "Warts Molluscum and other Viral Infections"]

In [15]:
label_to_remedy = {
    "Acne and Rosacea Photos": "Acne and Rosacea",
    "Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions": "Actinic Keratosis and Skin Cancers",
    "Atopic Dermatitis Photos": "Atopic Dermatitis (Eczema)",
    "Bullous Disease Photos": "Bullous Diseases",
    "Cellulitis Impetigo and other Bacterial Infections": "Cellulitis and Bacterial Infections",
    "Eczema Photos": "Atopic Dermatitis (Eczema)",
    "Exanthems and Drug Eruptions": "Exanthems and Drug Eruptions",
    "Hair Loss Photos Alopecia and other Hair Diseases": "Hair Loss (Alopecia)",
    "Herpes HPV and other STDs Photos": "Herpes and HPV",
    "Light Diseases and Disorders of Pigmentation": "Pigmentation Disorders",
    "Lupus and other Connective Tissue diseases": "Lupus and Connective Tissue Diseases",
    "Melanoma Skin Cancer Nevi and Moles": "Melanoma and Moles",
    "Nail Fungus and other Nail Disease": "Nail Fungus",
    "Poison Ivy Photos and other Contact Dermatitis": "Contact Dermatitis",
    "Psoriasis pictures Lichen Planus and related diseases": "Psoriasis and Lichen Planus",
    "Scabies Lyme Disease and other Infestations and Bites": "Scabies and Lyme Disease",
    "Seborrheic Keratoses and other Benign Tumors": "Seborrheic Keratoses",
    "Systemic Disease": "Systemic Diseases (Skin Signs)",
    "Tinea Ringworm Candidiasis and other Fungal Infections": "Fungal Infections (Tinea, Candidiasis)",
    "Urticaria Hives": "Urticaria (Hives)",
    "Vascular Tumors": "Vascular Tumors",
    "Vasculitis Photos": "Vasculitis",
    "Warts Molluscum and other Viral Infections": "Warts and Molluscum"
}

In [16]:
def predict_condition(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(128,128))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    predictions = model.predict(img_array)
    predicted_class = class_labels[np.argmax(predictions)]

    # Find remedy
    remedy_key = label_to_remedy.get(predicted_class, None)
    if remedy_key and remedy_key in remedies:
        remedy = remedies[remedy_key]
    else:
        remedy = "Remedy not available. Consult a doctor."

    return predicted_class, remedy

In [17]:
def predict_condition(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(128,128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    predictions = model.predict(img_array)
    predicted_class = class_labels[np.argmax(predictions)]

    # Map model label to unified key
    remedy_key = label_to_remedy.get(predicted_class, predicted_class)

    # Prefer detailed info from skin_diseases
    if remedy_key in skin_diseases:
        remedy = skin_diseases[remedy_key]  
    else:
        simple_remedies = globals().get("remedies", {})
        remedy = simple_remedies.get(predicted_class, "Remedy not available. Consult a doctor.")

    return predicted_class, remedy

In [20]:
file_path = "1.jpeg"  
condition, remedy = predict_condition(file_path)

print(f"Predicted Condition: {condition}")
print("\n--- Suggested Remedy ---")
if isinstance(remedy, dict):
    for k, v in remedy.items():
        print(f"{k}: {v}")
else:
    print(remedy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Predicted Condition: Eczema Photos

--- Suggested Remedy ---
Overview: Chronic inflammatory skin condition with flare-ups.
Causes: Genetics, immune dysfunction, allergens, climate.
Symptoms: Dry, itchy, red patches, flare-ups.
Care: Moisturizers, hydrocortisone cream (doctor), lukewarm baths, avoid harsh soaps/allergens.
